In [2]:
import json
import pandas as pd
import itertools
import csv
import pickle
import nltk

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD
from sklearn.decomposition import PCA

In [2]:
file_path = '/Users/xzhou/github/project_files/amazon/amazon_{}'

alexa_file = file_path.format('alexa.tsv')
echo_file = file_path.format('echo.csv')

In [3]:
# Load file #1. Parse date into date format

df_alexa = pd.read_csv(alexa_file, sep='\t', parse_dates=['date'])
df_alexa.head()

,rating,date,variation,verified_reviews,feedback
0,5,2018-07-31,Charcoal Fabric,Love my Echo!,1
1,5,2018-07-31,Charcoal Fabric,Loved it!,1
2,4,2018-07-31,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,2018-07-31,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,2018-07-31,Charcoal Fabric,Music,1


In [4]:
# Drop columns that won't be used in analysis

columns_to_drop = ['variation', 'feedback']
df_alexa.drop(columns_to_drop, axis=1, inplace=True)

In [5]:
df_alexa.rename(index=str, 
                columns={'verified_reviews': 'review'}, 
                inplace=True)
df_alexa.head()

,rating,date,review
0,5,2018-07-31,Love my Echo!
1,5,2018-07-31,Loved it!
2,4,2018-07-31,"Sometimes while playing a game, you can answer..."
3,5,2018-07-31,I have had a lot of fun with this thing. My 4 ...
4,5,2018-07-31,Music


In [6]:
# Check for null values

print (df_alexa.shape)
df_alexa.isnull().sum()

(3150, 3)


rating    0
date      0
review    0
dtype: int64

In [7]:
# Load file #2, parse 'Review Date' as date type

df_echo = pd.read_csv(echo_file, parse_dates=['Review Date'])
df_echo.head(5)

,Pageurl,Title,Review Text,Review Color,User Verified,Review Date,Review Useful Count,Configuration Text,Rating,Declaration Text
0,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Three Stars,Not great speakers,Black,Verified Purchase,2017-10-03,NaN,Echo Dot,3,NaN
1,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Four Stars,Great little gagit,White,Verified Purchase,2017-09-26,NaN,Echo Dot,4,NaN
2,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Awesome!,Awesome 👏🏽,White,Verified Purchase,2017-09-08,NaN,Echo Dot,5,NaN
3,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Five Stars,Love my Echo,Black,Verified Purchase,2017-10-19,NaN,Echo Dot,5,NaN
4,https://www.amazon.com/All-New-Amazon-Echo-Dot...,Five Stars,Great device,Black,Verified Purchase,2017-09-17,NaN,Echo Dot,5,NaN


In [8]:
# Drop columns that won't be used in analysis

columns_to_drop2 = ['Pageurl', 'Title','Review Color', 
                    'User Verified','Review Useful Count', 
                    'Configuration Text','Declaration Text']
df_echo.drop(columns_to_drop2, axis=1, inplace=True)
df_echo.head()

,Review Text,Review Date,Rating
0,Not great speakers,2017-10-03,3
1,Great little gagit,2017-09-26,4
2,Awesome 👏🏽,2017-09-08,5
3,Love my Echo,2017-10-19,5
4,Great device,2017-09-17,5


In [9]:
# Align the columns of 2 Dataframes. This is to parepare 2 Dataframes for concatenation

df_echo.insert(0, 'date', df_echo['Review Date'])
df_echo.insert(0, 'rating', df_echo['Rating'])
df_echo.drop(['Review Date', 'Rating'], axis=1, inplace=True)
df_echo.head()

,rating,date,Review Text
0,3,2017-10-03,Not great speakers
1,4,2017-09-26,Great little gagit
2,5,2017-09-08,Awesome 👏🏽
3,5,2017-10-19,Love my Echo
4,5,2017-09-17,Great device


In [10]:
df_echo.rename(index=str, 
                columns={'Review Text': 'review'}, 
                inplace=True)
df_echo.head()

,rating,date,review
0,3,2017-10-03,Not great speakers
1,4,2017-09-26,Great little gagit
2,5,2017-09-08,Awesome 👏🏽
3,5,2017-10-19,Love my Echo
4,5,2017-09-17,Great device


In [11]:
df_echo.shape

(6855, 3)

In [12]:
df_echo.isnull().sum()

rating    0
date      0
review    3
dtype: int64

In [13]:
# Drop null values in review

df_echo.dropna(inplace=True)
df_echo.isnull().sum()

rating    0
date      0
review    0
dtype: int64

In [14]:
# Merge two dataframes into one

df = pd.concat([df_alexa, df_echo])
df.head()

,rating,date,review
0,5,2018-07-31,Love my Echo!
1,5,2018-07-31,Loved it!
2,4,2018-07-31,"Sometimes while playing a game, you can answer..."
3,5,2018-07-31,I have had a lot of fun with this thing. My 4 ...
4,5,2018-07-31,Music


In [15]:
df.shape

(10002, 3)

In [17]:
# Break down reviews by word

df['review_by_word'] = df['review'].apply(nltk.word_tokenize)

In [21]:
# Perform a word count for each review. 
# This is to remove reviews that don't have minimum leangh

df['word_count'] = df['review_by_word'].apply(len)
df.head()

,rating,date,review,review_by_word,word_count
0,5,2018-07-31,Love my Echo!,"[Love, my, Echo, !]",4
1,5,2018-07-31,Loved it!,"[Loved, it, !]",3
2,4,2018-07-31,"Sometimes while playing a game, you can answer...","[Sometimes, while, playing, a, game, ,, you, c...",41
3,5,2018-07-31,I have had a lot of fun with this thing. My 4 ...,"[I, have, had, a, lot, of, fun, with, this, th...",38
4,5,2018-07-31,Music,[Music],1


In [23]:
# Drill down to comments that have meaningful length

df_smaller = df[df['word_count']>=5]
df_smaller.shape

(8347, 5)

In [25]:
# Drop duplicate reviews if there are any

df_smaller = df_smaller.drop_duplicates(['review'], keep='first')
df_smaller.shape

(7248, 5)

In [4]:
df_smaller.drop(['review_by_word', 'word_count'], axis=1, inplace=True)
df_smaller.head()

,rating,date,review
2,4,2018-07-31,"Sometimes while playing a game, you can answer..."
3,5,2018-07-31,I have had a lot of fun with this thing. My 4 ...
5,5,2018-07-31,I received the echo as a gift. I needed anothe...
6,3,2018-07-31,"Without having a cellphone, I cannot use many ..."
7,5,2018-07-31,I think this is the 5th one I've purchased. I'...


In [5]:
# Store data in pickle file for further analysis

pkl_path = '/Users/xzhou/github/project_files/amazon/amazon_alexa_reviews_final.pkl'

try:
    with open(pkl_path, 'rb') as pkl_file:
        df_smaller = pd.read_pickle(pkl_file)
except:
    with open(pkl_path, 'wb') as pkl_file:
        df_smaller.to_pickle(pkl_file)